# Import Libraries

In [37]:
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    FunctionTransformer,
    StandardScaler,
    MinMaxScaler,
    OneHotEncoder,
    OrdinalEncoder,
)
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn import set_config

import mlflow
import dagshub

from typing import Optional, List, Dict, Tuple

from ml.utils.utils import (
    get_data,
)

%matplotlib inline

# Dagshub Integration for MLflow Experiment Tracking

In [2]:
mlflow.is_tracking_uri_set()

False

In [3]:
dagshub.init(repo_owner='pramitde726', repo_name='Smartphone-Spec-Score-Analysis', mlflow=True)


Accessing as pramitde726

Initialized MLflow to track repo "pramitde726/Smartphone-Spec-Score-Analysis"

Repository pramitde726/Smartphone-Spec-Score-Analysis initialized!

In [6]:
mlflow.get_tracking_uri()

'https://dagshub.com/pramitde726/Smartphone-Spec-Score-Analysis.mlflow'

In [33]:
# Set the maximum number of columns to display
pd.set_option('display.max_columns', None) 

# Enable pandas output for the pipeline
set_config(transform_output="pandas")

## Ignore warnings
warnings.filterwarnings("ignore")

## Read the Data

In [24]:
df = get_data("smartphone_cleaned_v2", "processed")
final_df = remove_outliers(df).drop(columns = ["model"])
final_df.head()

,price,rating,os,brand_name,has_5g,has_nfc,has_ir_blaster,num_cores,processor_speed,processor_brand,...,screen_size,resolution,refresh_rate,num_rear_cameras,num_front_cameras,primary_camera_rear,primary_camera_front,fast_charging_available,extended_memory_available,extended_upto
0,54999,89.0,android,oneplus,True,True,False,8.0,3.2,snapdragon,...,6.70,1440 x 3216,120,3,1,50,16.0,1,0,0.0
1,19989,81.0,android,oneplus,True,False,False,8.0,2.2,snapdragon,...,6.59,1080 x 2412,120,3,1,64,16.0,1,1,1024.0
2,16499,75.0,android,samsung,True,False,False,8.0,2.4,exynos,...,6.60,1080 x 2408,90,3,1,50,13.0,1,1,1024.0
3,14999,81.0,android,motorola,True,False,False,8.0,2.2,snapdragon,...,6.55,1080 x 2400,120,3,1,50,16.0,1,1,1024.0
4,24999,82.0,android,realme,True,False,False,8.0,2.6,dimensity,...,6.70,1080 x 2412,120,3,1,108,16.0,1,0,0.0


## Train Test Split

In [25]:
cols_to_drop = ["os", "battery_capacity"]

In [26]:
X = final_df.dropna(subset=["rating"]).drop(columns=cols_to_drop).drop(columns='rating')
y = final_df['rating'].dropna()

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [28]:
print("The size of train data is",X_train.shape)
print("The shape of test data is",X_test.shape)

The size of train data is (685, 21)
The shape of test data is (172, 21)


## Schema

In [17]:
standard_scale_cols = [
    'price', 'processor_speed', 'screen_size',
    'performance_score', 'camera_quality', 'storage_expandability'
]

minmax_scale_cols = ['connectivity_features']

nominal_cols = ['brand_name', 'processor_brand']

ordinal_cols = [
    "extended_upto",
    "has_5g",
    "has_nfc",
    "has_ir_blaster",
    "num_cores",
    "ram_capacity",
    "internal_memory",
    "fast_charging",
    "resolution",
    "refresh_rate",
    "num_rear_cameras",
    "primary_camera_rear",
    "primary_camera_front",
    "fast_charging_available",
    "extended_memory_available",
]

## Data Preprocessing 

### Imputation Pipeline

In [3]:
class DropNA(BaseEstimator, TransformerMixin):
    """
    A transformer that drops rows with missing values (NaN) from a Pandas DataFrame.

    :param subset: List of columns to consider when dropping rows with NaN values.
                   If None, all columns are considered.
    :type subset: Optional[List[str]], optional

    Attributes
    ----------
    subset : Optional[List[str]]
        List of columns to consider when dropping rows with NaN values.
    """
    def __init__(self, subset: Optional[List[str]] = None):
        self.subset = subset

    def fit(self, X: pd.DataFrame, y: Optional[pd.Series] = None) -> "DropNA":
        """
        Fit the transformer. This method does not actually perform any computation.

        :param X: The input DataFrame.
        :type X: pd.DataFrame
        :param y: Ignored. Present here for compatibility with the scikit-learn API.
        :type y: Optional[pd.Series], optional
        :return: Returns self.
        :rtype: DropNA
        """
        return self

    def transform(self, X: pd.DataFrame, y: Optional[pd.Series] = None) -> pd.DataFrame:
        """
        Drop rows with missing values (NaN) from the input DataFrame.

        :param X: The input DataFrame.
        :type X: pd.DataFrame
        :param y: Ignored. Present here for compatibility with the scikit-learn API.
        :type y: Optional[pd.Series], optional
        :return: The DataFrame with rows containing NaN values in the specified subset of columns removed.
        :rtype: pd.DataFrame
        """
        return X.dropna(subset=self.subset)

In [4]:
class ProcessorBrandImputer(BaseEstimator, TransformerMixin):
    """
    Imputes missing 'processor_brand' values in a DataFrame based on the 'brand_name'.

    This transformer learns the most frequent 'processor_brand' for each 'brand_name'
    during the `fit` method and uses this information to impute missing values
    during the `transform` method. Handles cases where a brand has no associated
    processor brand in the training data, or where a specific brand ('ikall' or 'tesla')
    should always have an 'Unknown' processor.

    Attributes
    ----------
    brand_processor_mode : Dict[str, str]
        A dictionary mapping 'brand_name' to the most frequent 'processor_brand'.
        Stores the mode for each brand name learned during the fit step.
    """
    def __init__(self):
        self.brand_processor_mode: Optional[Dict[str, str]] = None

    def fit(self, X: pd.DataFrame, y: Optional[pd.Series] = None) -> "ProcessorBrandImputer":
        """
        Learns the most frequent 'processor_brand' for each 'brand_name'.

        Calculates the mode of 'processor_brand' for each 'brand_name' present
        in the input DataFrame `X` and stores it in `self.brand_processor_mode`.

        :param X: The input DataFrame containing 'brand_name' and 'processor_brand' columns.
        :type X: pd.DataFrame
        :param y: Ignored. Present here for compatibility with the scikit-learn API.
        :type y: Optional[pd.Series], optional
        :return: Returns self.
        :rtype: ProcessorBrandImputer
        """
        self.brand_processor_mode = (
            X[X['processor_brand'].notnull()]
            .groupby('brand_name')['processor_brand']
            .agg(lambda x: x.mode()[0] if not x.mode().empty else 'Unknown')
            .to_dict()
        )
        return self

    def transform(self, X: pd.DataFrame, y: Optional[pd.Series] = None) -> pd.DataFrame:
        """
        Imputes missing 'processor_brand' values in the input DataFrame.

        Uses the learned `self.brand_processor_mode` to fill missing
        'processor_brand' values. If a 'brand_name' is not found in the
        learned mapping, 'Unknown' is used as the imputation value. Also handles
        special cases where 'ikall' and 'tesla' brands are always assigned
        the 'Unknown' processor.

        :param X: The input DataFrame containing 'brand_name' and 'processor_brand' columns.
        :type X: pd.DataFrame
        :param y: Ignored. Present here for compatibility with the scikit-learn API.
        :type y: Optional[pd.Series], optional
        :return: The DataFrame with missing 'processor_brand' values imputed.
        :rtype: pd.DataFrame
        """
        def impute_processor(row):
            if row['brand_name'] in ['ikall', 'tesla']:
                return 'Unknown'
            elif pd.isnull(row['processor_brand']):
                return self.brand_processor_mode.get(row['brand_name'], 'Unknown')
            return row['processor_brand']

        X['processor_brand'] = X.apply(impute_processor, axis=1)
        return X

In [5]:
class BatteryCapacityImputer(BaseEstimator, TransformerMixin):
    """
    Imputes missing 'battery_capacity' values with the median battery capacity
    of Apple devices.

    This transformer calculates the median 'battery_capacity' for devices where
    'brand_name' (case-insensitive) is 'apple' during the `fit` method and uses
    this median to fill missing 'battery_capacity' values during the `transform`
    method.

    Attributes
    ----------
    apple_median : Optional[float]
        The median battery capacity of Apple devices.
    """
    def __init__(self):
        self.apple_median: Optional[float] = None

    def fit(self, X: pd.DataFrame, y: Optional[pd.Series] = None) -> "BatteryCapacityImputer":
        """
        Calculates the median battery capacity for Apple devices.

        :param X: The input DataFrame containing 'brand_name' and 'battery_capacity' columns.
        :type X: pd.DataFrame
        :param y: Ignored. Present here for compatibility with the scikit-learn API.
        :type y: Optional[pd.Series], optional
        :return: Returns self.
        :rtype: BatteryCapacityImputer
        """
        self.apple_median = X.loc[X['brand_name'].str.lower() == 'apple', 'battery_capacity'].median()
        return self

    def transform(self, X: pd.DataFrame, y: Optional[pd.Series] = None) -> pd.DataFrame:
        """
        Imputes missing 'battery_capacity' values with the calculated median.

        :param X: The input DataFrame containing 'battery_capacity' column.
        :type X: pd.DataFrame
        :param y: Ignored. Present here for compatibility with the scikit-learn API.
        :type y: Optional[pd.Series], optional
        :return: The DataFrame with missing 'battery_capacity' values imputed.
        :rtype: pd.DataFrame
        """
        X['battery_capacity'] = X['battery_capacity'].fillna(self.apple_median)
        return X


class ExtendedUptoImputer(BaseEstimator, TransformerMixin):
    """
    Imputes missing 'extended_upto' values with the median 'extended_upto' value
    for devices where 'extended_memory_available' is 1.

    Attributes
    ----------
    extended_memory_median : Optional[float]
        The median 'extended_upto' value for devices with extended memory available.
    """
    def __init__(self):
        self.extended_memory_median: Optional[float] = None

    def fit(self, X: pd.DataFrame, y: Optional[pd.Series] = None) -> "ExtendedUptoImputer":
        """
        Calculates the median 'extended_upto' value for devices with extended memory.

        :param X: The input DataFrame containing 'extended_memory_available' and 'extended_upto' columns.
        :type X: pd.DataFrame
        :param y: Ignored. Present here for compatibility with the scikit-learn API.
        :type y: Optional[pd.Series], optional
        :return: Returns self.
        :rtype: ExtendedUptoImputer
        """
        self.extended_memory_median = X.loc[X['extended_memory_available'] == 1, 'extended_upto'].median()
        return self

    def transform(self, X: pd.DataFrame, y: Optional[pd.Series] = None) -> pd.DataFrame:
        """
        Imputes missing 'extended_upto' values.

        :param X: The input DataFrame containing 'extended_upto' column.
        :type X: pd.DataFrame
        :param y: Ignored. Present here for compatibility with the scikit-learn API.
        :type y: Optional[pd.Series], optional
        :return: The DataFrame with missing 'extended_upto' values imputed.
        :rtype: pd.DataFrame
        """
        X['extended_upto'] = X.apply(
            lambda row: row['extended_upto'] if pd.notnull(row['extended_upto'])
            else self.extended_memory_median,
            axis=1
        )
        return X


class FastChargingImputer(BaseEstimator, TransformerMixin):
    """
    Imputes missing 'fast_charging' values to 0 when 'fast_charging_available' is 0.
    """
    def fit(self, X: pd.DataFrame, y: Optional[pd.Series] = None) -> "FastChargingImputer":
        """
        Fit the transformer. This method does not actually perform any computation.

        :param X: The input DataFrame.
        :type X: pd.DataFrame
        :param y: Ignored. Present here for compatibility with the scikit-learn API.
        :type y: Optional[pd.Series], optional
        :return: Returns self.
        :rtype: FastChargingImputer
        """
        return self

    def transform(self, X: pd.DataFrame, y: Optional[pd.Series] = None) -> pd.DataFrame:
        """
        Imputes 'fast_charging' values.

        Sets 'fast_charging' to 0 if 'fast_charging_available' is 0 and 'fast_charging' is NaN.

        :param X: The input DataFrame containing 'fast_charging' and 'fast_charging_available' columns.
        :type X: pd.DataFrame
        :param y: Ignored. Present here for compatibility with the scikit-learn API.
        :type y: Optional[pd.Series], optional
        :return: The DataFrame with imputed 'fast_charging' values.
        :rtype: pd.DataFrame
        """
        X['fast_charging'] = X.apply(
            lambda row: 0 if row['fast_charging_available'] == 0 and pd.isnull(row['fast_charging'])
            else row['fast_charging'],
            axis=1
        )
        return X

In [6]:
class ConsistentBrandsImputer(BaseEstimator, TransformerMixin):
    """
    Imputes missing 'fast_charging' values for specific brands with their medians.

    This transformer calculates the median 'fast_charging' value for a predefined
    list of brands ('lg', 'oneplus', 'xiaomi', 'vivo') during the `fit` method.
    It then uses these medians to impute missing 'fast_charging' values for the
    respective brands during the `transform` method.

    Attributes
    ----------
    brand_medians : Dict[str, Optional[float]]
        A dictionary storing the median 'fast_charging' value for each
        consistent brand.
    """
    def __init__(self):
        self.brand_medians: Dict[str, Optional[float]] = {}  # Store median fast_charging for each brand

    def fit(self, X: pd.DataFrame, y: Optional[pd.Series] = None) -> "ConsistentBrandsImputer":
        """
        Calculates the median 'fast_charging' for consistent brands.

        :param X: The input DataFrame containing 'brand_name' and 'fast_charging' columns.
        :type X: pd.DataFrame
        :param y: Ignored. Present here for compatibility with the scikit-learn API.
        :type y: Optional[pd.Series], optional
        :return: Returns self.
        :rtype: ConsistentBrandsImputer
        """
        consistent_brands = ['lg', 'oneplus', 'xiaomi', 'vivo']
        for brand in consistent_brands:
            median_value = X.loc[X['brand_name'].str.lower() == brand, 'fast_charging'].median()
            self.brand_medians[brand] = median_value
        return self

    def transform(self, X: pd.DataFrame, y: Optional[pd.Series] = None) -> pd.DataFrame:
        """
        Imputes missing 'fast_charging' values for consistent brands.

        :param X: The input DataFrame containing 'brand_name' and 'fast_charging' columns.
        :type X: pd.DataFrame
        :param y: Ignored. Present here for compatibility with the scikit-learn API.
        :type y: Optional[pd.Series], optional
        :return: The DataFrame with imputed 'fast_charging' values for consistent brands.
        :rtype: pd.DataFrame
        """
        for brand, median_value in self.brand_medians.items():
            X.loc[
                (X['brand_name'].str.lower() == brand) & (X['fast_charging'].isnull()),
                'fast_charging'
            ] = median_value
        return X

In [7]:
class HigherMissingBrandsImputer(BaseEstimator, TransformerMixin):
    """
    Imputes missing 'fast_charging' values for specific brands with their medians.

    This transformer calculates the median 'fast_charging' value for a predefined
    list of brands ('samsung', 'apple') during the `fit` method. It then uses
    these medians to impute missing 'fast_charging' values for the respective
    brands during the `transform` method.

    Attributes
    ----------
    brand_medians : Dict[str, Optional[float]]
        A dictionary storing the median 'fast_charging' value for each
        specified brand.
    """
    def __init__(self):
        self.brand_medians: Dict[str, Optional[float]] = {}

    def fit(self, X: pd.DataFrame, y: Optional[pd.Series] = None) -> "HigherMissingBrandsImputer":
        """
        Calculates the median 'fast_charging' for specified brands.

        :param X: The input DataFrame containing 'brand_name' and 'fast_charging' columns.
        :type X: pd.DataFrame
        :param y: Ignored. Present here for compatibility with the scikit-learn API.
        :type y: Optional[pd.Series], optional
        :return: Returns self.
        :rtype: HigherMissingBrandsImputer
        """
        higher_missing_brands = ['samsung', 'apple']
        for brand in higher_missing_brands:
            median_value = X.loc[X['brand_name'].str.lower() == brand, 'fast_charging'].median()
            self.brand_medians[brand] = median_value
        return self

    def transform(self, X: pd.DataFrame, y: Optional[pd.Series] = None) -> pd.DataFrame:
        """
        Imputes missing 'fast_charging' values for specified brands.

        :param X: The input DataFrame containing 'brand_name' and 'fast_charging' columns.
        :type X: pd.DataFrame
        :param y: Ignored. Present here for compatibility with the scikit-learn API.
        :type y: Optional[pd.Series], optional
        :return: The DataFrame with imputed 'fast_charging' values for specified brands.
        :rtype: pd.DataFrame
        """
        for brand, median_value in self.brand_medians.items():
            X.loc[
                (X['brand_name'].str.lower() == brand) & (X['fast_charging'].isnull()),
                'fast_charging'
            ] = median_value
        return X

In [8]:
class LowerMissingBrandsImputer(BaseEstimator, TransformerMixin):
    """
    Imputes missing 'fast_charging' values for specific brands with their medians.

    This transformer calculates the median 'fast_charging' value for a predefined
    list of brands ('sony', 'xiaomi', 'oppo', 'motorola', 'google') during the
    `fit` method. It then uses these medians to impute missing 'fast_charging'
    values for the respective brands during the `transform` method.

    Attributes
    ----------
    brand_medians : Dict[str, Optional[float]]
        A dictionary storing the median 'fast_charging' value for each
        specified brand.
    """
    def __init__(self):
        self.brand_medians: Dict[str, Optional[float]] = {}

    def fit(self, X: pd.DataFrame, y: Optional[pd.Series] = None) -> "LowerMissingBrandsImputer":  
        """
        Calculates the median 'fast_charging' for specified brands.

        :param X: The input DataFrame containing 'brand_name' and 'fast_charging' columns.
        :type X: pd.DataFrame
        :param y: Ignored. Present here for compatibility with the scikit-learn API.
        :type y: Optional[pd.Series], optional
        :return: Returns self.
        :rtype: LowerMissingBrandsImputer 
        """
        lower_missing_brands = ['sony', 'xiaomi', 'oppo', 'motorola', 'google']
        for brand in lower_missing_brands:
            median_value = X.loc[X['brand_name'].str.lower() == brand, 'fast_charging'].median()
            self.brand_medians[brand] = median_value
        return self

    def transform(self, X: pd.DataFrame, y: Optional[pd.Series] = None) -> pd.DataFrame:
        """
        Imputes missing 'fast_charging' values for specified brands.

        :param X: The input DataFrame containing 'brand_name' and 'fast_charging' columns.
        :type X: pd.DataFrame
        :param y: Ignored. Present here for compatibility with the scikit-learn API.
        :type y: Optional[pd.Series], optional
        :return: The DataFrame with imputed 'fast_charging' values for specified brands.
        :rtype: pd.DataFrame
        """
        for brand, median_value in self.brand_medians.items():
            X.loc[
                (X['brand_name'].str.lower() == brand) & (X['fast_charging'].isnull()),
                'fast_charging'
            ] = median_value
        return X

In [9]:
class SingleRecordBrandsRemover(BaseEstimator, TransformerMixin):
    """
    Removes records associated with specific brands from a DataFrame.

    This transformer removes rows where the 'brand_name' (case-insensitive) is
    present in a predefined or user-provided list of brands.

    Attributes
    ----------
    brands_to_remove : List[str], default=['blu', 'leitz', 'sharp']
        A list of brand names to remove from the DataFrame.
    """
    def __init__(self, brands_to_remove: Optional[List[str]] = None):
        self.brands_to_remove = brands_to_remove or ['blu', 'leitz', 'sharp']  # Default brands

    def fit(self, X: pd.DataFrame, y: Optional[pd.Series] = None) -> "SingleRecordBrandsRemover":
        """
        Fit the transformer. This method does not actually perform any computation.

        :param X: The input DataFrame.
        :type X: pd.DataFrame
        :param y: Ignored. Present here for compatibility with the scikit-learn API.
        :type y: Optional[pd.Series], optional
        :return: Returns self.
        :rtype: SingleRecordBrandsRemover
        """
        return self

    def transform(self, X: pd.DataFrame, y: Optional[pd.Series] = None) -> pd.DataFrame:
        """
        Removes records associated with the specified brands.

        :param X: The input DataFrame containing the 'brand_name' column.
        :type X: pd.DataFrame
        :param y: Ignored. Present here for compatibility with the scikit-learn API.
        :type y: Optional[pd.Series], optional
        :return: The DataFrame with records associated with the specified brands removed.
        :rtype: pd.DataFrame
        """
        return X[~X['brand_name'].str.lower().isin(self.brands_to_remove)]

In [10]:
class OSByBrandImputer(BaseEstimator, TransformerMixin):
    """
    Imputes missing 'os' values based on the mode 'os' for each 'brand_name'.

    This transformer calculates the mode of the 'os' column for each
    'brand_name' during the `fit` method and uses these modes to impute
    missing 'os' values during the `transform` method.

    Attributes
    ----------
    os_mode : Dict[str, Optional[str]]
        A dictionary mapping each 'brand_name' to its most frequent 'os' value.
        Can be None if fit has not been called.
    """
    def __init__(self):
        self.os_mode: Optional[Dict[str, Optional[str]]] = None

    def fit(self, X: pd.DataFrame, y: Optional[pd.Series] = None) -> "OSByBrandImputer":  # String literal
        """
        Calculates the mode 'os' for each 'brand_name'.

        :param X: The input DataFrame containing 'brand_name' and 'os' columns.
        :type X: pd.DataFrame
        :param y: Ignored. Present here for compatibility with the scikit-learn API.
        :type y: Optional[pd.Series], optional
        :return: Returns self.
        :rtype: OSByBrandImputer
        """
        self.os_mode = X.groupby('brand_name')['os'].agg(lambda x: x.mode()[0] if not x.mode().empty else None).to_dict()
        return self

    def transform(self, X: pd.DataFrame, y: Optional[pd.Series] = None) -> pd.DataFrame:
        """
        Imputes missing 'os' values based on the calculated modes.

        :param X: The input DataFrame containing 'brand_name' and 'os' columns.
        :type X: pd.DataFrame
        :param y: Ignored. Present here for compatibility with the scikit-learn API.
        :type y: Optional[pd.Series], optional
        :return: The DataFrame with imputed 'os' values.
        :rtype: pd.DataFrame
        """
        X['os'] = X.apply(
            lambda row: self.os_mode.get(row['brand_name']) if pd.isnull(row['os']) else row['os'], axis=1
        )
        return X

In [11]:
# Handle missing values pipeline
missing_handler = Pipeline([
    ('camera_dropna', DropNA(subset=["primary_camera_front"])),
    ('processor_brand', ProcessorBrandImputer()),
    ('num_cores_dropna', DropNA(subset=["num_cores"])),
    ('processor_speed_dropna', DropNA(subset=["processor_speed"])),
    ('extended_upto', ExtendedUptoImputer()),
    ('fast_charging',  FastChargingImputer()),
    ('consistent_brands', ConsistentBrandsImputer()),
    ('higher_missing_brands', HigherMissingBrandsImputer()),
    ('lower_missing_brands', LowerMissingBrandsImputer()),
    ('single_record_brands', SingleRecordBrandsRemover()),

])
    

### Feature Construction 

In [12]:
def compute_performance_score(X: pd.DataFrame) -> pd.DataFrame:
    """
    Computes a performance score based on processor speed, number of cores, and RAM capacity.

    :param X: The input DataFrame containing 'processor_speed', 'num_cores', and 'ram_capacity' columns.
    :type X: pd.DataFrame
    :return: The DataFrame with the added 'performance_score' column.
    :rtype: pd.DataFrame
    """
    X = X.copy()
    X["performance_score"] = (
        X["processor_speed"] * X["num_cores"].astype(float) * X["ram_capacity"].astype(float)
    )
    return X

def compute_battery_efficiency(X: pd.DataFrame) -> pd.DataFrame:
    """
    Computes battery efficiency based on battery capacity and screen size.

    :param X: The input DataFrame containing 'battery_capacity' and 'screen_size' columns.
    :type X: pd.DataFrame
    :return: The DataFrame with the added 'battery_efficiency' column.
    :rtype: pd.DataFrame
    """
    X = X.copy()
    X["battery_efficiency"] = X["battery_capacity"] / X["screen_size"]
    return X

def compute_camera_quality(X: pd.DataFrame) -> pd.DataFrame:
    """
    Computes camera quality based on rear and front camera megapixels.

    :param X: The input DataFrame containing 'primary_camera_rear' and 'primary_camera_front' columns.
    :type X: pd.DataFrame
    :return: The DataFrame with the added 'camera_quality' column.
    :rtype: pd.DataFrame
    """
    X = X.copy()
    X["camera_quality"] = X["primary_camera_rear"].astype(float) + X["primary_camera_front"].astype(float)
    return X

def compute_connectivity_features(X: pd.DataFrame) -> pd.DataFrame:
    """
    Computes a connectivity features score based on 5g, NFC, and IR blaster availability.

    :param X: The input DataFrame containing 'has_5g', 'has_nfc', and 'has_ir_blaster' columns.
    :type X: pd.DataFrame
    :return: The DataFrame with the added 'connectivity_features' column.
    :rtype: pd.DataFrame
    """
    X = X.copy()
    X["connectivity_features"] = X[["has_5g", "has_nfc", "has_ir_blaster"]].sum(axis=1)
    return X

def compute_storage_expandability(X: pd.DataFrame) -> pd.DataFrame:
    """
    Computes storage expandability based on maximum expandable memory and availability.

    :param X: The input DataFrame containing 'extended_upto' and 'extended_memory_available' columns.
    :type X: pd.DataFrame
    :return: The DataFrame with the added 'storage_expandability' column.
    :rtype: pd.DataFrame
    """
    X = X.copy()
    X["storage_expandability"] = X["extended_upto"].astype(float) * X["extended_memory_available"].astype(int)
    return X

In [13]:
# feature construction pipeline

feature_constructor = Pipeline([
    ("performance_score", FunctionTransformer(compute_performance_score)),
    ("camera_quality", FunctionTransformer(compute_camera_quality)),
    ("connectivity_features", FunctionTransformer(compute_connectivity_features)),
    ("storage_expandability", FunctionTransformer(compute_storage_expandability)),
])

### High Cardinality 

In [14]:
def encode_features(X: pd.DataFrame) -> pd.DataFrame:
    """
    Encodes categorical and numerical features into discrete categories.

    This function performs binning and grouping operations on several features
    to convert them into categorical representations. It handles 'resolution',
    'brand_name', 'ram_capacity', 'processor_brand', 'fast_charging',
    'primary_camera_rear', 'primary_camera_front', and 'extended_upto'.

    :param X: The input DataFrame.
    :type X: pd.DataFrame
    :return: The DataFrame with encoded features.
    :rtype: pd.DataFrame
    """
    X = X.copy()

    # resolution
    X['resolution'] = np.select(
        [
            X["resolution"].str.extract(r"(\d+)").astype(float).iloc[:, 0] < 1080,
            (X["resolution"].str.extract(r"(\d+)").astype(float).iloc[:, 0] >= 1080) &
            (X["resolution"].str.extract(r"(\d+)").astype(float).iloc[:, 0] < 1440),
            (X["resolution"].str.extract(r"(\d+)").astype(float).iloc[:, 0] >= 1440) &
            (X["resolution"].str.extract(r"(\d+)").astype(float).iloc[:, 0] < 2160),
            X["resolution"].str.extract(r"(\d+)").astype(float).iloc[:, 0] >= 2160,
        ],
        ["HD Ready", "Full HD", "Quad HD", "Ultra HD"],
        default="Unknown"
    )

    # brand_name
    X['brand_name'] = X['brand_name'].mask(X['brand_name'].map(X['brand_name'].value_counts()) < 10, "other")

    # ram_capacity
    X['ram_capacity'] = np.select(
        [
            X["ram_capacity"].astype(float) <= 4,
            X["ram_capacity"].astype(float) <= 8
        ],
        ["Low", "Standard"],
        default="High"
    )

    # processor_brand
    X['processor_brand'] = X['processor_brand'].mask(X['processor_brand'].map(X['processor_brand'].value_counts()) < 5, "other")

    # fast_charging
    X['fast_charging'] = np.select(
        [
            X["fast_charging"] == 0.0,
            (X["fast_charging"] > 0.0) & (X["fast_charging"] <= 18),
            (X["fast_charging"] > 18) & (X["fast_charging"] <= 33),
            (X["fast_charging"] > 33) & (X["fast_charging"] <= 65),
            (X["fast_charging"] > 65) & (X["fast_charging"] <= 120)
        ],
        ["no", "basic", "moderate", "fast", "ultra fast"],
        default="extreme fast"
    )

    # primary_camera_rear
    X['primary_camera_rear'] = np.select(
        [
            X["primary_camera_rear"].astype(float) <= 8,
            (X["primary_camera_rear"].astype(float) > 8) & (X["primary_camera_rear"].astype(float) <= 16),
            (X["primary_camera_rear"].astype(float) > 16) & (X["primary_camera_rear"].astype(float) <= 48),
            (X["primary_camera_rear"].astype(float) > 48) & (X["primary_camera_rear"].astype(float) <= 64),
            (X["primary_camera_rear"].astype(float) > 64) & (X["primary_camera_rear"].astype(float) <= 108)
        ],
        ["basic", "standard", "good", "high-Resolution", "ultra-high"],
        default="extreme"
    )

    # primary_camera_front
    X['primary_camera_front'] = np.select(
        [
            X["primary_camera_front"].astype(float) <= 8,
            (X["primary_camera_front"].astype(float) > 8) & (X["primary_camera_front"].astype(float) <= 16),
            (X["primary_camera_front"].astype(float) > 16) & (X["primary_camera_front"].astype(float) <= 32),
            (X["primary_camera_front"].astype(float) > 32) & (X["primary_camera_front"].astype(float) <= 48)
        ],
        ["basic", "standard", "good", "high-resolution"],
        default="ultra high"
    )

    # extended_upto
    X['extended_upto'] = np.select(
        [
            X["extended_upto"] == 0.0,
            X["extended_upto"].isin([64.0, 128.0, 256.0]),
            X["extended_upto"] == 512.0,
            X["extended_upto"] == 1024.0,
            X["extended_upto"] == 2048.0
        ],
        ["no expansion", "small", "medium", "large", "very large"],
        default="unknown"
    )

    return X

### Feature Encoding

In [15]:
# generate order for ordinal encoding

ordinal_categories = [
    ["no expansion", "small", "medium", "large", "very large", "unknown"],  # extended_upto
    [False, True],  # has_5g
    [False, True],  # has_nfc
    [False, True],  # has_ir_blaster
    [4.0, 6.0, 8.0],  # num_cores
    ["Low", "Standard", "High"],  # ram_capacity
    [8.0, 16.0, 32.0, 64.0, 128.0, 256.0, 512.0, 1024.0],  # internal_memory
    ["no", "basic", "moderate", "fast", "ultra fast", "extreme fast"],  # fast_charging
    ["HD Ready", "Full HD", "Quad HD", "Ultra HD"],  # resolution
    [60, 90, 120, 144, 165],  # refresh_rate
    [1, 2, 3, 4],  # num_rear_cameras
    ["basic", "standard", "good", "high-Resolution", "ultra-high", "extreme"],  # primary_camera_rear
    ["basic", "standard", "good", "high-resolution", "ultra high"],  # primary_camera_front
    [0, 1],  # fast_charging_available
    [0, 1]   # extended_memory_available
]


In [18]:
feature_encoder = ColumnTransformer([
    ('standard_scaling', StandardScaler(), standard_scale_cols),
    ('minmax_scaling', MinMaxScaler(), minmax_scale_cols),
    ('nominal_encoding', OneHotEncoder(handle_unknown='ignore', sparse_output=False), nominal_cols),
    ('ordinal_encoding', OrdinalEncoder(categories=ordinal_categories, 
                                        handle_unknown="use_encoded_value", 
                                        unknown_value=-1), ordinal_cols)
])


### Preprocessor Pipeline

In [19]:
preprocessor = Pipeline([
    ('handle_missing', missing_handler),
    ('feature_construction', feature_constructor),
    ('high_cardinality', FunctionTransformer(encode_features)),
    ('feature_encoding', feature_encoder)  
])

In [20]:
preprocessor

Pipeline(steps=[('handle_missing',
                 Pipeline(steps=[('camera_dropna',
                                  DropNA(subset=['primary_camera_front'])),
                                 ('processor_brand', ProcessorBrandImputer()),
                                 ('num_cores_dropna',
                                  DropNA(subset=['num_cores'])),
                                 ('processor_speed_dropna',
                                  DropNA(subset=['processor_speed'])),
                                 ('extended_upto', ExtendedUptoImputer()),
                                 ('fast_charging', FastChargingImputer()),
                                 ('cons...
                                                                              'high-resolution',
                                                                              'ultra '
                                                                              'high'],
                                                                             [0,
                                                                              1],
                                                                             [0,
                                                                              1]],
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['extended_upto', 'has_5g',
                                                   'has_nfc', 'has_ir_blaster',
                                                   'num_cores', 'ram_capacity',
                                                   'internal_memory',
                                                   'fast_charging',
                                                   'resolution', 'refresh_rate',
                                                   'num_rear_cameras',
                                                   'primary_camera_rear',
                                                   'primary_camera_front',
                                                   'fast_charging_available',
                                                   'extended_memory_available'])]))])

In [21]:
preprocessor.get_params(deep=True)

{'memory': None,
 'steps': [('handle_missing',
   Pipeline(steps=[('camera_dropna', DropNA(subset=['primary_camera_front'])),
                   ('processor_brand', ProcessorBrandImputer()),
                   ('num_cores_dropna', DropNA(subset=['num_cores'])),
                   ('processor_speed_dropna', DropNA(subset=['processor_speed'])),
                   ('extended_upto', ExtendedUptoImputer()),
                   ('fast_charging', FastChargingImputer()),
                   ('consistent_brands', ConsistentBrandsImputer()),
                   ('higher_missing_brands', HigherMissingBrandsImputer()),
                   ('lower_missing_brands', LowerMissingBrandsImputer()),
                   ('single_record_brands',
                    SingleRecordBrandsRemover(brands_to_remove=['blu', 'leitz',
                                                                'sharp']))])),
  ('feature_construction',
   Pipeline(steps=[('performance_score',
                    FunctionTransformer(fun

In [34]:
X_train_trans = preprocessor.fit_transform(X_train)
X_test_trans = preprocessor.transform(X_test)

In [35]:
y_train_trans = y_train.loc[X_train_trans.index]
y_test_trans = y_test.loc[X_test_trans.index]

### Baseline model

#### Linear Regression

In [41]:
lr = LinearRegression()
lr.fit(X_train_trans, y_train_trans)

LinearRegression()

In [42]:
# get the predictions
y_pred_train = lr.predict(X_train_trans)
y_pred_test = lr.predict(X_test_trans)

In [43]:
# calculate the cross val score

scores = cross_val_score(lr,
                         X_train_trans,
                         y_train_trans,
                         cv=5,scoring="r2",
                         n_jobs=-1)

print(scores)
print(f"Mean cross val score: {scores.mean():.2f}")

[0.91116131 0.93644466 0.93109249 0.92202914 0.94625314]
Mean cross val score: 0.93


In [44]:
# mae and r2 score
print(f"The train error is {mean_absolute_error(y_train_trans,y_pred_train):.2f}")
print(f"The test error is {mean_absolute_error(y_test_trans,y_pred_test):.2f}")

print(f"The train r2 score is {r2_score(y_train_trans,y_pred_train):.2f}")
print(f"The test r2 score is {r2_score(y_test_trans,y_pred_test):.2f}")

The train error is 1.42
The test error is 1.73
The train r2 score is 0.94
The test r2 score is 0.90


In [45]:
# mlflow experiment

mlflow.set_experiment("Exp 1 - Baseline Model")


<Experiment: artifact_location='mlflow-artifacts:/0c40bdf6844f4d8b94e0a96fdd0d011c', creation_time=1739372362026, experiment_id='1', last_update_time=1739372362026, lifecycle_stage='active', name='Exp 1 - Baseline Model', tags={}>

In [38]:
def plotResidue(X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series, 
                y_test: pd.Series, y_pred_train: pd.Series, y_pred_test: pd.Series, 
                output_dir: str = "mlflow_artifacts/") -> Tuple[str, float, float]:
    """
    Plots the residuals (actual - predicted values) vs. the first numerical feature in X_train.
    Saves the figure and returns its path. Also computes and returns mean residual and standard deviation.

    :param X_train: Pandas DataFrame, training features
    :type X_train: pd.DataFrame
    :param X_test: Pandas DataFrame, test features
    :type X_test: pd.DataFrame
    :param y_train: Pandas Series, actual training target
    :type y_train: pd.Series
    :param y_test: Pandas Series, actual test target
    :type y_test: pd.Series
    :param y_pred_train: Pandas Series, predicted training target
    :type y_pred_train: pd.Series
    :param y_pred_test: Pandas Series, predicted test target
    :type y_pred_test: pd.Series
    :param output_dir: str, directory to save the figure (default: 'mlflow_artifacts/')
    :type output_dir: str, optional
    :return: tuple: (fig_path, mean_residual, std_residual)
    :rtype: Tuple[str, float, float]
    """

    # Compute residuals
    residue_train = y_train - y_pred_train
    residue_test = y_test - y_pred_test

    # Compute mean residual and standard deviation
    mean_residual = round(np.mean(residue_test), 2)
    std_residual = round(np.std(residue_test), 2)

    # Select first numerical column from X_train for visualization
    numeric_cols = X_train.select_dtypes(include=np.number).columns
    if not numeric_cols.empty:
        feature_name = numeric_cols[0]
    else:
        raise ValueError("X_train does not contain any numerical features.")


    # Create subplots
    fig, axe = plt.subplots(1, 2, figsize=(18, 10))

    # Residuals scatter plot
    axe[0].scatter(X_train[feature_name], residue_train, label="Train", alpha=0.7)
    axe[0].scatter(X_test[feature_name], residue_test, label="Test", alpha=0.7)
    axe[0].axhline(y=0, color="black", linestyle="dashed", linewidth=2)  # Horizontal line at 0
    axe[0].set_xlabel(f"{feature_name} (Feature)")
    axe[0].set_ylabel("Residuals (Actual - Predicted)")
    axe[0].legend()
    axe[0].set_title("Residuals vs Feature Plot")

    # Residuals histogram
    axe[1].hist(residue_test, bins=25, alpha=0.75, color="blue", edgecolor="black")
    axe[1].set_xlabel("Residuals (Test Set)")
    axe[1].set_ylabel("Count")
    axe[1].set_title("Distribution of Residuals")

    # Define output path using pathlib
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)  # Create directory if it doesn't exist
    fig_path = output_dir / "residual_plot.png"

    # Save the figure
    plt.savefig(fig_path)
    plt.close(fig)

    # Return the figure path along with residual statistics
    return str(fig_path), mean_residual, std_residual

In [47]:
plotResidue(
    X_train = X_train_trans,
    X_test = X_test_trans,
    y_train = y_train_trans,
    y_test = y_test_trans,
    y_pred_train = y_pred_train,
    y_pred_test = y_pred_test
)

('mlflow_artifacts\\residual_plot.png', np.float64(-0.04), np.float64(2.27))

In [48]:
# log experiment
with mlflow.start_run(run_name="Linear Regression"):
    # mlflow log experiment type
    mlflow.log_param("experiment_type", "Linear Regression")
    
    # log model params
    mlflow.log_params(lr.get_params())

    # log metrics
    mlflow.log_metric("training_error", mean_absolute_error(y_train_trans,y_pred_train))
    mlflow.log_metric("test_error", mean_absolute_error(y_test_trans,y_pred_test))
    mlflow.log_metric("training_r2", r2_score(y_train_trans,y_pred_train))
    mlflow.log_metric("test_r2", r2_score(y_test_trans,y_pred_test))
    mlflow.log_metric("cross_val", scores.mean())

    # Generate and save residual plot
    fig_path, mean_residual, std_residual = plotResidue(
        X_train=X_train_trans,
        X_test=X_test_trans,
        y_train=y_train_trans,
        y_test=y_test_trans,
        y_pred_train=y_pred_train,
        y_pred_test=y_pred_test
    )

    # Log residual statistics as parameters
    mlflow.log_metric("mean_residual", mean_residual)
    mlflow.log_metric("std_residual", std_residual)

    # Explicitly log the residual plot in MLflow
    mlflow.log_artifact(fig_path)


🏃 View run Linear Regression at: https://dagshub.com/pramitde726/Smartphone-Spec-Score-Analysis.mlflow/#/experiments/1/runs/da36fe76a3bf4945b56c88312e081c0e
🧪 View experiment at: https://dagshub.com/pramitde726/Smartphone-Spec-Score-Analysis.mlflow/#/experiments/1


In [49]:
# Get coefficients
coefficients = lr.coef_

# Feature names (replace with your actual feature names)
feature_names = X_train_trans.columns.to_list()

# Plotting
plt.figure(figsize=(10, 6))
plt.bar(feature_names, coefficients)
plt.xlabel("Features")
plt.ylabel("Coefficient Value")
plt.title("Feature Coefficients in Linear Regression")
plt.xticks(rotation=45, ha="right") 
plt.tight_layout() 
plt.grid(axis='y', linestyle='--', alpha=0.7) 
plt.show()


#### Decision Tree

In [65]:
dt = DecisionTreeRegressor(criterion="absolute_error")

dt.fit(X_train_trans, y_train_trans)

DecisionTreeRegressor(criterion='absolute_error')

In [66]:
# get the predictions
y_pred_train = dt.predict(X_train_trans)
y_pred_test = dt.predict(X_test_trans)


In [67]:
# calculate the cross val score

scores = cross_val_score(dt,
                         X_train_trans,
                         y_train_trans,
                         cv=5,scoring="r2",
                         n_jobs=-1)

print(scores)
print(f"Mean cross val score: {scores.mean():.2f}")


[0.90593917 0.88482253 0.91765931 0.88178668 0.92841703]
Mean cross val score: 0.90


In [68]:
# mae and r2 score
print(f"The train error is {mean_absolute_error(y_train_trans,y_pred_train):.2f}")
print(f"The test error is {mean_absolute_error(y_test_trans,y_pred_test):.2f}")

print(f"The train r2 score is {r2_score(y_train_trans,y_pred_train):.2f}")
print(f"The test r2 score is {r2_score(y_test_trans,y_pred_test):.2f}")

The train error is 0.00
The test error is 1.80
The train r2 score is 1.00
The test r2 score is 0.87


In [69]:
# Compute residuals
residue_train = y_train_trans - y_pred_train
residue_test = y_test_trans - y_pred_test

mean_residual = round(np.mean(residue_test), 2)
std_residual = round(np.std(residue_test), 2)

In [70]:
# log experiment
with mlflow.start_run(run_name="Decision Tree"):
    # mlflow log experiment type
    mlflow.log_param("experiment_type", "Decision Tree")
    
    # log model params
    mlflow.log_params(dt.get_params())

    # log metrics
    mlflow.log_metric("training_error", mean_absolute_error(y_train_trans,y_pred_train))
    mlflow.log_metric("test_error", mean_absolute_error(y_test_trans,y_pred_test))
    mlflow.log_metric("training_r2", r2_score(y_train_trans,y_pred_train))
    mlflow.log_metric("test_r2", r2_score(y_test_trans,y_pred_test))
    mlflow.log_metric("cross_val", scores.mean())

    # Compute mean residual and standard deviation
    mlflow.log_metric("mean_residual", float(mean_residual))  
    mlflow.log_metric("std_residual", float(std_residual))  

🏃 View run Decision Tree at: https://dagshub.com/pramitde726/Smartphone-Spec-Score-Analysis.mlflow/#/experiments/1/runs/bedba371f37949bfadf972a38fbc0020
🧪 View experiment at: https://dagshub.com/pramitde726/Smartphone-Spec-Score-Analysis.mlflow/#/experiments/1


In [71]:
# feature importance plot

(
    pd.DataFrame(dt.feature_importances_,
             index=X_train_trans.columns,
             columns=["importance"])
    .sort_values(by="importance")
    .plot(kind='barh',figsize=(10,10))
)
   

<Axes: >

## Observations

- Decision Tree is performing better and exceeds minimum metrics requirements.
- Feature selection and hyperparamter tuning will be done.